In [5]:
import gzip
import json
import re

class PropDbReader:
    def __init__(self, ids_path, offsets_path, avs_path, attrs_path, vals_path):
        with gzip.open(ids_path, 'rb') as ids_file, gzip.open(offsets_path, 'rb') as offsets_file, \
             gzip.open(avs_path, 'rb') as avs_file, gzip.open(attrs_path, 'rb') as attrs_file, \
             gzip.open(vals_path, 'rb') as vals_file:
            self.ids = json.load(ids_file)
            self.offsets = json.load(offsets_file)
            self.avs = json.load(avs_file)
            self.attrs = json.load(attrs_file)
            self.vals = json.load(vals_file)

    def enumerateProperties(self, id):
        properties = []
        if 0 < id < len(self.offsets):
            av_start = 2 * self.offsets[id]
            av_end = len(self.avs) if id == len(self.offsets) - 1 else 2 * self.offsets[id + 1]
            for i in range(av_start, av_end, 2):
                attr_offset = self.avs[i]
                val_offset = self.avs[i + 1]
                attr_obj = self.attrs[attr_offset]

                # Check if attr_obj is a list and has at least two elements
                if isinstance(attr_obj, list) and len(attr_obj) >= 2:
                    name = attr_obj[0]
                    category = attr_obj[1]
                else:
                    name = "N/A"
                    category = "N/A"

                value = self.vals[val_offset]
                properties.append(Property(name, category, value))
        return properties


    def getProperties(self, id):
        props = {}
        rg = re.compile(r'^__\w+__$')
        for prop in self.enumerateProperties(id):
            if prop.category and not rg.match(prop.category):
                props[prop.name] = prop.value
        return props

    def getPropertiesByCategory(self, id):
        properties = {}
        rg = re.compile(r'^__\w+__$')
        categories = []

        props = self.enumerateProperties(id)
        for prop in props:
            if prop.category:
                if not rg.match(prop.category):
                    if not any(category in prop.category for category in categories):
                        categories.append(prop.category)

        for category in categories:
            prop_result = [prop for prop in props if prop.category == category]
            prop_dictionary = []
            for prop in prop_result:
                prop_key = prop.name
                prop_dictionary.append((prop_key, prop.value))
            properties[category] = prop_dictionary

        return properties

    def getChildren(self, id):
        children = []
        for prop in self.enumerateProperties(id):
            if prop.category == "__child__":
                children.append(int(prop.value))
        return children
    def getParent(self, id):
        parent = []
        for prop in self.enumerateProperties(id):
            if prop.category == "__parent__":
                parent.append(int(prop.value))
        return parent
    def getInstance(self, id):
        instanceOf = []
        for prop in self.enumerateProperties(id):
            if prop.category == "__instanceof__":
                instanceOf.append(int(prop.value))
        return instanceOf

class Property:
    def __init__(self, name, category, value):
        self.name = name
        self.category = category
        self.value = value


In [6]:
# Create a sample data (in byte format) for the PropDbReader constructor
object_att = 'svfs/objects_attrs.json.gz'
object_val = 'svfs/objects_vals.json.gz'
object_avs = 'svfs/objects_avs.json.gz'
object_ids = 'svfs/objects_ids.json.gz'
objects_offs = 'svfs/objects_offs.json.gz'

# Create an instance of PropDbReader with the sample data
prop_db = PropDbReader(object_ids, objects_offs, object_avs, object_att, object_val)
# Try Get Properties Chair at location idndex is 3528 with InternalId : 5bb069ca-e4fe-4e63-be31-f8ac44e80d30-00046bfe
# InternalId = UniqueId of Element, in this case we try with a chair from Revit Model have UniqueId = 5bb069ca-e4fe-4e63-be31-f8ac44e80d30-00046bfe
id_index = 3528
prop_db.getProperties(id_index)

{'IFC Predefined Type': '',
 'Export to IFC As': '',
 'Export to IFC': 'By Type',
 'IfcGUID': '1Ri6dAvFvEOxun_An4x6RE',
 'Level': 'Level 1',
 'Elevation from Level': 0,
 'Host': 'Level : Level 1',
 'Offset from Host': 0,
 'Moves With Nearby Elements': 0,
 'Phase Created': 'Project Completion',
 'Phase Demolished': 'None',
 'Armrest Left': 0,
 'Armrest Right': 0,
 'Image': '',
 'Comments': 'This is a chair',
 'Mark': '',
 'Workset': 'Workset1',
 'Edited by': '',
 'Type Name': 'Plastic-Seat',
 'Room Name': 'Sleep',
 'Room Number': 'A001'}

In [7]:
# How to Get Parameter Type of Element
typeIndex = prop_db.getInstance(id_index)
# the output is [3527] which is the type of element, we will try to get the properties of this type
prop_db.getProperties(typeIndex[0])

{'Type IFC Predefined Type': '',
 'Export Type to IFC As': '',
 'Export Type to IFC': 'Default',
 'Type IfcGUID': '1Ri6dAvFvEOxun_An4x6RO',
 'Default Elevation': 0,
 'Seat Material': 'Plastic - Black',
 'Legs Material': 'Plastic - Black',
 'Frame Material': 'Colour - Coated - Black',
 'Back Material': 'Plastic - Black',
 'Width': 510,
 'Weight': '4.1',
 'Height': 790,
 'Depth': 500,
 'URL': 'www.lammhults.se',
 'Model': 'PENNE',
 'Manufacturer': 'LAMMHULTS',
 'Keynote': '46.B',
 'Description': '*Please enter mtrl description here for use in mto*',
 'Content Author': 'AEC AB',
 'Assembly Code': 'E2020',
 'Type Image': '',
 'Type Comments': '',
 'Cost': 0,
 'Assembly Description': 'Moveable Furnishings',
 'Type Mark': '',
 'Workset': 'Family  : Furniture : Seating-LAMMHULTS-PENNE-Chair',
 'Edited by': '',
 'OmniClass Number': '23.40.20.00',
 'OmniClass Title': 'General Furniture and Specialties',
 'Code Name': ''}

In [8]:
# this one include more information of categories of properties
prop_db.getPropertiesByCategory(id_index)

{'IFC Parameters': [('IFC Predefined Type', ''),
  ('Export to IFC As', ''),
  ('Export to IFC', 'By Type'),
  ('IfcGUID', '1Ri6dAvFvEOxun_An4x6RE')],
 'Constraints': [('Level', 'Level 1'),
  ('Elevation from Level', 0),
  ('Host', 'Level : Level 1'),
  ('Offset from Host', 0),
  ('Moves With Nearby Elements', 0)],
 'Phasing': [('Phase Created', 'Project Completion'),
  ('Phase Demolished', 'None')],
 'Construction': [('Armrest Left', 0), ('Armrest Right', 0)],
 'Identity Data': [('Image', ''),
  ('Comments', 'This is a chair'),
  ('Mark', ''),
  ('Workset', 'Workset1'),
  ('Edited by', ''),
  ('Type Name', 'Plastic-Seat')],
 'Other': [('Room Name', 'Sleep'), ('Room Number', 'A001')]}